# PTESOL Listening TTS — Google Colab Notebook

This notebook runs the PTESOL listening audio generator using Bark TTS on Google Colab (GPU recommended).

Usage notes:
- Select Runtime > Change runtime type > GPU
- Colab GPU memory and disk limits may affect model loading — use the test cell before running the full generation.
- The notebook expects the SQL files containing the listening questions (two files) to be uploaded or available in your Drive.
- Voice used: `v2/en_speaker_3` (English Male — Casual & Friendly)

Sections:
1. Install dependencies
2. Import and setup
3. Mount Google Drive
4. Upload or point to SQL files
5. Parse SQL questions
6. Generate conversation scripts
7. Load Bark model
8. Test generation (1–3 samples)
9. Full generation (all questions)
10. Summary and upload guidance

Press SHIFT+ENTER to run cells.


In [ ]:
# Section 1 — Install Required Dependencies

# Run these in Colab. They install transformers, bark (if available), and audio libs.
# If you already have torch on Colab GPU, the install will skip/upgrade as needed.

# Recommended: use a GPU runtime (Runtime -> Change runtime type -> GPU)

!pip install -q --upgrade pip
!pip install -q transformers accelerate scipy numpy soundfile git+https://github.com/suno-ai/bark || true
# Install a compatible torch build for the GPU in Colab (this may already be present)
!python -c "import torch; print('Torch', torch.__version__, 'CUDA', torch.version.cuda)"
!nvidia-smi || true

print('\n✅ Install step finished. If any package failed, check the output and re-run the cell.')

In [ ]:
# Section 2 — Import Libraries and Setup Device

import os
import re
from datetime import datetime
import numpy as np
import scipy.io.wavfile

import torch
from transformers import AutoProcessor, BarkModel

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)

# Voice preset
VOICE_PRESET = 'v2/en_speaker_3'
print('Voice preset:', VOICE_PRESET)

# Default local output folder (will be created on first run)
LOCAL_OUTPUT_DIR = '/content/ptesol_listening_audio'
os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)
print('Local output dir:', LOCAL_OUTPUT_DIR)


In [ ]:
# Section 3 — Mount Google Drive (optional but recommended)

# If you want to save generated audio to Google Drive, mount it here.
# You can skip this if you prefer to download the ZIP later.

from google.colab import drive

drive.mount('/content/drive')

# Recommended Drive folder (create if needed)
DRIVE_OUTPUT_DIR = '/content/drive/MyDrive/ptesol_listening_audio'
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)
print('Drive output dir:', DRIVE_OUTPUT_DIR)
